In [132]:
from sklearn.manifold import MDS
import numpy as np
import torch
import pandas as pd
import random
from matplotlib import pyplot as plt

from sklearn.metrics import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
import utils as ut
import TumorDecon as td


In [2]:
J = 10 #number of patients
N = 100 #dimensions of clinical data
b = np.random.random_sample(size=(J,J))*2-1
S= (b + b.T)/2

In [3]:
np.fill_diagonal(S, 0)

# Start from the basics
### From a similarity matrix get some points with that similarity

In [4]:
og_points=np.random.poisson(size=[J,N])
matrix=euclidean_distances(og_points,og_points)

In [5]:
# perform MDS
X_mds = ut.fake_points_MDS(matrix,N)


/home/federico/.local/lib/python3.10/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [6]:
matrix_empirical=euclidean_distances(X_mds,X_mds)

In [7]:
np.sum(matrix_empirical-matrix)

0.2687542480913283

# Forward model

In [8]:
J = 10 #number of patients
N = 30 #dimensions of clinical data
C = 3 #number of cell types
G = 100 #genes number
sc_samples=1000 #samples to create signature matrix

In [9]:
features=np.random.normal(scale=1/np.sqrt(N),size=[J,N])
weights=np.random.normal(scale=1/np.sqrt(N),size=[N,G])

In [10]:
patient_correction=np.exp(np.matmul(features,weights))
#get a genexpatient
patient_correction = patient_correction.T
cell_type_correction=np.exp(np.random.normal(scale=0.5,size=[G,C]))

In [11]:
print(patient_correction.shape)

(100, 10)


In [12]:
shape=10
scale=1
average_mu=np.random.gamma(shape,scale,size=[G])
# Multiply M and V, broadcasting happens automatically
average_mu = average_mu[:, np.newaxis]

# Broadcast multiplication
mu_cell_type = average_mu * cell_type_correction
mu = mu_cell_type[:, :, np.newaxis] * patient_correction[:, np.newaxis, :]
print(mu.shape)

(100, 3, 10)


In [13]:
T_torch = torch.from_numpy(mu)

# Use the Poisson function to generate samples
sc_counts= torch.poisson(T_torch.unsqueeze(-1).expand(*T_torch.shape, sc_samples))

# Now I have to create the mixtures
# permute the dimensions to have patients- cell types-samples-genes
sc_counts = sc_counts.permute(2, 1, 3, 0)


In [14]:
cell_type_profiles = torch.mean(sc_counts, dim=(0,2))
cell_type_profiles_per_patient=torch.mean(sc_counts,dim=2)
bulk_counts_per_patient=torch.sum(sc_counts, dim=(1,2))

In [15]:
#tumordecon requires to use some real genes names
# Load the data
df = pd.read_csv('genes_list.txt', sep='\t')

# Extract the 'symbol' column, which contains the gene names
gene_names = df['Approved symbol'].tolist()

# Choose 5 random gene names
random_gene_names = random.sample(gene_names, 100)
genes=random_gene_names
genes_entrez=["gene " + str(i) for i in range(1, 101)]
patients = ["patient " + str(i) for i in range(1, 11)]

In [16]:
# Create a DataFrame
df = pd.DataFrame(cell_type_profiles.T, index=genes, columns=["cell type 1", "cell type 2" , "cell type 3"])
df.index.name="Gene_Symbol"
df=df.reset_index()
#df['Entrez_Gene_Id']=genes_entrez
# Save the DataFrame to a CSV file
#df.to_csv("signature_matrix.csv")
df.to_csv('data/signature_matrix.tsv', sep='\t',index=False)

In [17]:
df

,Gene_Symbol,cell type 1,cell type 2,cell type 3
0,UNC93A,19.6975,9.8995,9.7818
1,CARF,17.1865,8.0320,4.8773
2,TMEM51,12.6813,10.4007,8.1513
3,CHD3,6.7244,23.6381,15.4479
4,TMPRSS15,16.6235,7.1544,11.6604
...,...,...,...,...
95,BMAL2,11.4714,10.2990,6.3702
96,ICOS,14.0140,7.9541,5.9810
97,CYP26C1,8.9449,6.4547,10.9240
98,ANKUB1,16.5187,6.9436,17.9952


In [18]:
# Create a DataFrame
df = pd.DataFrame(cell_type_profiles_per_patient[0].T, index=genes, columns=["cell type 1", "cell type 2" , "cell type 3"])

# Save the DataFrame to a CSV file
df.to_csv("signature_matrix_one_patient.csv")

In [19]:
df

,cell type 1,cell type 2,cell type 3
UNC93A,18.388,9.357,9.145
CARF,19.139,9.020,5.276
TMEM51,10.188,8.642,6.529
CHD3,5.312,19.138,12.157
TMPRSS15,12.525,5.326,8.746
...,...,...,...
BMAL2,9.304,8.145,5.066
ICOS,11.820,6.777,5.008
CYP26C1,9.372,6.663,11.507
ANKUB1,14.264,6.066,15.705


In [20]:
# Create a DataFrame
df = pd.DataFrame(bulk_counts_per_patient.T, index=genes, columns=patients)
df.index.name="Hugo_Symbol"
df=df.reset_index()
df['Entrez_Gene_Id']=genes_entrez
#df.set_index('Entrez_Gene_Id', inplace=True)
# Save the DataFrame to a CSV file
#df.to_csv("data/bulk_counts.csv")
df.to_csv('data/bulk_counts.tsv', sep='\t',index=False)

# Try TumorDecon

### As in the tutorial in https://people.math.umass.edu/~aronow/TumorDecon/quickstart.html#tutorial

In [21]:
# Location of sample data (included with the TumorDecon package):
data_loc = "./data/"
# Read in sample data (original source - Colorectal Adenocarcinoma RNA Seq v2 from cBioPortal.org):
rna = td.read_rna_file(data_loc+'bulk_counts.tsv',identifier='hugo',fetch_missing_hugo=False)


In [22]:
signature=td.read_sig_file(data_loc+"signature_matrix.tsv")

In [23]:
ciber_freqs = td.tumor_deconvolve(rna, 'cibersort',  patient_IDs='ALL', sig_matrix=signature, args={'nu':'best', 'scaling':'minmax'})


Running CiberSort...
CiberSort has completed!


In [24]:
ciber_freqs

Patient_ID,cell type 1,cell type 2,cell type 3
patient 1,0.571606,0.209404,0.218990
patient 2,0.441771,0.232256,0.325973
patient 3,0.546242,0.209249,0.244509
patient 4,0.451778,0.255085,0.293137
patient 5,0.484127,0.210884,0.304989
patient 6,0.395172,0.256195,0.348634
patient 7,0.442825,0.237669,0.319506
patient 8,0.560836,0.212379,0.226785
patient 9,0.460823,0.270636,0.268541
patient 10,0.426937,0.229529,0.343534


# Mix them together -> Try to fake a pipeline 

In [104]:
#generate a fake distance matrix
J = 10 #number of patients
N = 100 #dimensions of clinical data
b = np.random.random_sample(size=(J,J))
S= (b + b.T)/2
np.fill_diagonal(S, 0)

In [128]:
patient0=np.random.normal(scale=1/np.sqrt(N),size=N)
patients=[]
patients.append(patient0)
for i in range(1,J):
    patient=np.random.normal(scale=1/np.sqrt(N),size=N)+patients[-1]
    patients.append(patient)
patients=np.array(patients)

In [107]:
weights=np.random.normal(scale=1/np.sqrt(N),size=[N,G])
patient_correction=np.exp(np.matmul(patients,weights))
#get a genexpatient
patient_correction = patient_correction.T
print(patient_correction.shape)
cell_type_correction=np.exp(np.random.normal(scale=1,size=[G,C]))

(100, 10)


In [108]:
shape=10
scale=1
average_mu=np.random.gamma(shape,scale,size=[G])
# Multiply M and V, broadcasting happens automatically
average_mu = average_mu[:, np.newaxis]

# Broadcast multiplication
mu_cell_type = average_mu * cell_type_correction
#Alternative:
mu_cell_type=np.random.gamma(shape,scale,size=[G,C])
print(mu_cell_type.shape)
mu = mu_cell_type[:, :, np.newaxis] * patient_correction[:, np.newaxis, :]
print(mu.shape)

(100, 3)
(100, 3, 10)


In [109]:
T_torch = torch.from_numpy(mu)

# Use the Poisson function to generate samples
sc_counts= torch.poisson(T_torch.unsqueeze(-1).expand(*T_torch.shape, sc_samples))

# Now I have to create the mixtures
# permute the dimensions to have patients- cell types-samples-genes
sc_counts = sc_counts.permute(2, 1, 3, 0)


In [110]:
cell_type_profiles = torch.mean(sc_counts, dim=(0,2))
cell_type_profiles_per_patient=torch.mean(sc_counts,dim=2)
bulk_counts_per_patient=torch.sum(sc_counts, dim=(1,2))

In [111]:
#tumordecon requires to use some real genes names
# Load the data
df = pd.read_csv('genes_list.txt', sep='\t')

# Extract the 'symbol' column, which contains the gene names
gene_names = df['Approved symbol'].tolist()

# Choose 5 random gene names
random_gene_names = random.sample(gene_names, 100)
genes=random_gene_names
genes_entrez=["gene " + str(i) for i in range(1, 101)]
patients = ["patient " + str(i) for i in range(1, 11)]

In [112]:
# Create a DataFrame
df = pd.DataFrame(cell_type_profiles.T, index=genes, columns=["cell type 1", "cell type 2" , "cell type 3"])
df.index.name="Gene_Symbol"
df=df.reset_index()
#df['Entrez_Gene_Id']=genes_entrez
# Save the DataFrame to a CSV file
#df.to_csv("signature_matrix.csv")
df.to_csv('data/signature_matrix.tsv', sep='\t',index=False)

In [113]:
for i in range(len(cell_type_profiles_per_patient)):
    df = pd.DataFrame(cell_type_profiles_per_patient[i].T, index=genes, columns=["cell type 1", "cell type 2" , "cell type 3"])
    df.index.name="Gene_Symbol"
    df=df.reset_index()
    #df['Entrez_Gene_Id']=genes_entrez
    # Save the DataFrame to a CSV file
    #df.to_csv("signature_matrix.csv")
    df.to_csv(f'data/signature_matrix_{i}.tsv', sep='\t',index=False)

In [114]:
# Create a DataFrame
df = pd.DataFrame(bulk_counts_per_patient.T, index=genes, columns=patients)
df.index.name="Hugo_Symbol"
df=df.reset_index()
df['Entrez_Gene_Id']=genes_entrez
#df.set_index('Entrez_Gene_Id', inplace=True)
# Save the DataFrame to a CSV file
#df.to_csv("data/bulk_counts.csv")
df.to_csv('data/bulk_counts.tsv', sep='\t',index=False)

## Average signature matrix, as before

In [115]:
# Location of sample data (included with the TumorDecon package):
data_loc = "./data/"
# Read in sample data (original source - Colorectal Adenocarcinoma RNA Seq v2 from cBioPortal.org):
rna = td.read_rna_file(data_loc+'bulk_counts.tsv',identifier='hugo',fetch_missing_hugo=False)
signature=td.read_sig_file(data_loc+"signature_matrix.tsv")

In [116]:
ciber_freqs = td.tumor_deconvolve(rna, 'cibersort',  patient_IDs='ALL', sig_matrix=signature, args={'nu':'best', 'scaling':'minmax'})

Running CiberSort...
CiberSort has completed!


In [117]:
true_freqs=np.array([1/3,1/3,1/3])

In [118]:
euclidean_distances(ciber_freqs.to_numpy(),true_freqs.reshape(1, -1))

array([[0.43524018],
       [0.32816018],
       [0.12247741],
       [0.1575312 ],
       [0.13341881],
       [0.08876931],
       [0.1398508 ],
       [0.18278871],
       [0.12820191],
       [0.0916212 ]])

## Different signature matrices

In [119]:
signatures=[]
for i in range(J):
    signature=td.read_sig_file(data_loc+f"signature_matrix_{i}.tsv")
    signatures.append(signature)

In [126]:
distances=np.empty((J, J))
for i in range(J):
    ciber_freqs = td.tumor_deconvolve(rna, 'cibersort',  patient_IDs='ALL', sig_matrix=signatures[i], args={'nu':'best', 'scaling':'minmax'})
    dist=euclidean_distances(ciber_freqs.to_numpy(),true_freqs.reshape(1, -1))
    distances[i]=dist.flatten()

Running CiberSort...
CiberSort has completed!
Running CiberSort...
CiberSort has completed!
Running CiberSort...
CiberSort has completed!
Running CiberSort...
CiberSort has completed!
Running CiberSort...
CiberSort has completed!
Running CiberSort...
CiberSort has completed!
Running CiberSort...
CiberSort has completed!
Running CiberSort...
CiberSort has completed!
Running CiberSort...
CiberSort has completed!
Running CiberSort...
CiberSort has completed!


In [127]:
np.array(distances)

array([[0.05246967, 0.09240446, 0.03785011, 0.18892898, 0.03302898,
        0.07371174, 0.01322836, 0.17391841, 0.2084416 , 0.31452743],
       [0.11718825, 0.0263978 , 0.0437265 , 0.06840246, 0.13516875,
        0.0724545 , 0.13159936, 0.2724517 , 0.18732914, 0.28950964],
       [0.30868809, 0.12094095, 0.04716803, 0.02424421, 0.09416662,
        0.05384289, 0.0413261 , 0.17910279, 0.4088257 , 0.4138501 ],
       [0.3221176 , 0.23994356, 0.14159559, 0.15186714, 0.1874639 ,
        0.1219679 , 0.24137018, 0.24059675, 0.22943108, 0.17015387],
       [0.41133541, 0.35136472, 0.05623356, 0.08303235, 0.10237038,
        0.07764113, 0.28376307, 0.26316282, 0.28932854, 0.35757278],
       [0.41858842, 0.39495325, 0.20280985, 0.18587646, 0.1847737 ,
        0.10125668, 0.17453684, 0.16924481, 0.22234382, 0.12954549],
       [0.4254591 , 0.4785428 , 0.45227857, 0.3474785 , 0.24352985,
        0.07173481, 0.1328618 , 0.14444137, 0.13127176, 0.14011246],
       [0.51153585, 0.45722223, 0.4157479

In [122]:
ciber_freqs = td.tumor_deconvolve(rna, 'cibersort',  patient_IDs='ALL', sig_matrix=signatures[0], args={'nu':'best', 'scaling':'minmax'})

Running CiberSort...
CiberSort has completed!


In [123]:
ciber_freqs

Patient_ID,cell type 1,cell type 2,cell type 3
patient 1,0.294011,0.338269,0.367720
patient 2,0.407126,0.310050,0.282824
patient 3,0.309863,0.327657,0.362480
patient 4,0.353449,0.455728,0.190823
patient 5,0.345234,0.348341,0.306425
patient 6,0.288625,0.390581,0.320794
patient 7,0.322752,0.336749,0.340499
patient 8,0.235720,0.292823,0.471457
patient 9,0.179732,0.473608,0.346659
patient 10,0.079255,0.492730,0.428015
